In [6]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler



In [7]:
# Read the META.csv file from the Resources folder into a Pandas DataFrame
# Set the `Date` column as the DateTimeIndex
META_df = pd.read_csv(
    Path("../Resources/META.csv"), 
    index_col="Date",
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
display(META_df.head())
display(META_df.tail())

C:\Users\quaku\AppData\Local\Temp\ipykernel_16688\3827751842.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  META_df = pd.read_csv(


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-19,141.210007,144.910004,132.500000,133.240005,133.240005,57404900
2018-12-20,130.699997,135.570007,130.000000,133.399994,133.399994,40297900
2018-12-21,133.389999,134.899994,123.419998,124.949997,124.949997,56901500
2018-12-24,123.099998,129.740005,123.019997,124.059998,124.059998,22066000
2018-12-26,126.000000,134.240005,125.889999,134.179993,134.179993,39723400


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-12,324.600006,334.470001,324.559998,334.220001,334.220001,18485500
2023-12-13,333.929993,338.369995,332.640015,334.739990,334.739990,16353300
2023-12-14,333.850006,334.700012,328.640015,333.170013,333.170013,19607300
2023-12-15,331.989990,338.660004,331.220001,334.920013,334.920013,30001600
2023-12-18,337.480011,347.559998,337.019989,344.619995,344.619995,18993900


In [16]:
# Filter the date index and close columns
META_df = META_df.loc[:,["Close"]]
# Drop all NaN values from the DataFrame
META_df = META_df.dropna()

# Review the DataFrame
display(META_df.head())
display(META_df.tail())

,Close
Date,
2018-12-19,133.240005
2018-12-20,133.399994
2018-12-21,124.949997
2018-12-24,124.059998
2018-12-26,134.179993


,Close
Date,
2023-12-12,334.220001
2023-12-13,334.739990
2023-12-14,333.170013
2023-12-15,334.920013
2023-12-18,344.619995


In [17]:
def calculate_bollinger_bands(META_df, window=40, num_std_dev=2):
    META_df['Rolling Mean'] = META_df['Close'].rolling(window=window).mean()
    META_df['Upper Band'] = META_df['Rolling Mean'] + (META_df['Close'].rolling(window=window).std() * num_std_dev)
    META_df['Lower Band'] = META_df['Rolling Mean'] - (META_df['Close'].rolling(window=window).std() * num_std_dev)
    return META_df


In [22]:
print(calculate_bollinger_bands)

<function calculate_bollinger_bands at 0x000002468CFEFD80>


In [20]:
def plot_bollinger_bands(META_df):
    plt.figure(figsize=(12, 6))
    plt.plot(META_df['Close'], label='Close Price', color='blue')
    plt.plot(META_df['Rolling Mean'], label='Rolling Mean', color='black')
    plt.plot(META_df['Upper Band'], label='Upper Band', color='red', linestyle='dashed')
    plt.plot(META_df['Lower Band'], label='Lower Band', color='green', linestyle='dashed')
    plt.fill_between(META_df.index, META_df['Upper Band'], META_df['Lower Band'], color='gray', alpha=0.2)
    plt.title('Bollinger Bands Strategy')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()


In [21]:
def bollinger_band_strategy(META_df):
    signals = pd.DataFrame(index=META_df.index)
    signals['Signal'] = 0.0

    # Buy Signal
    signals['Signal'][META_df['Close'] < META_df['Lower Band']] = 1.0

    # Sell Signal
    signals['Signal'][META_df['Close'] > META_df['Upper Band']] = -1.0

    return signals


In [25]:
 # Calculate Bollinger Bands
stock_data = calculate_bollinger_bands(META_df)
stock_data

,Close,Rolling Mean,Upper Band,Lower Band
Date,,,,
2018-12-19,133.240005,NaN,NaN,NaN
2018-12-20,133.399994,NaN,NaN,NaN
2018-12-21,124.949997,NaN,NaN,NaN
2018-12-24,124.059998,NaN,NaN,NaN
2018-12-26,134.179993,NaN,NaN,NaN
...,...,...,...,...
2023-12-12,334.220001,321.878500,348.114412,295.642589
2023-12-13,334.739990,322.147000,348.690014,295.603986
2023-12-14,333.170013,322.552000,349.264770,295.839231


In [34]:
stock_data = stock_data.dropna()
stock_data

,Close,Rolling Mean,Upper Band,Lower Band
Date,,,,
2019-02-15,162.500000,148.354750,175.812597,120.896902
2019-02-19,162.289993,149.080999,176.435254,121.726745
2019-02-20,162.559998,149.809999,177.003551,122.616447
2019-02-21,160.039993,150.687249,176.834506,124.539992
2019-02-22,161.889999,151.632999,176.536061,126.729938
...,...,...,...,...
2023-12-12,334.220001,321.878500,348.114412,295.642589
2023-12-13,334.739990,322.147000,348.690014,295.603986
2023-12-14,333.170013,322.552000,349.264770,295.839231


In [36]:
signals = bollinger_band_strategy(META_df) 
signals

,Signal
Date,
2018-12-19,0.0
2018-12-20,0.0
2018-12-21,0.0
2018-12-24,0.0
2018-12-26,0.0
...,...
2023-12-12,0.0
2023-12-13,0.0
2023-12-14,0.0


In [40]:
stock_data["Signals"] = (bollinger_band_strategy(META_df)) 
stock_data

,Close,Rolling Mean,Upper Band,Lower Band,Signals
Date,,,,,
2019-02-15,162.500000,148.354750,175.812597,120.896902,0.0
2019-02-19,162.289993,149.080999,176.435254,121.726745,0.0
2019-02-20,162.559998,149.809999,177.003551,122.616447,0.0
2019-02-21,160.039993,150.687249,176.834506,124.539992,0.0
2019-02-22,161.889999,151.632999,176.536061,126.729938,0.0
...,...,...,...,...,...
2023-12-12,334.220001,321.878500,348.114412,295.642589,0.0
2023-12-13,334.739990,322.147000,348.690014,295.603986,0.0
2023-12-14,333.170013,322.552000,349.264770,295.839231,0.0


In [41]:
stock_data['Signals'].value_counts()

Signals
 0.0    1051
-1.0     101
 1.0      67
Name: count, dtype: int64

In [8]:
# Split the data into X (features) and y (lables)

# The y variable should focus on the Default column
y = META_df['Close']

# The X variable should include all features except the Default column
X = META_df.drop(columns=['Close'])


In [9]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\quaku\anaconda3\envs\mlenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\quaku\anaconda3\envs\mlenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\quaku\anaconda3\envs\mlenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\quaku\anaconda3\envs\mlenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

In [11]:
# Count the distinct values in the original labels data
y_train.value_counts()

Close
302.549988    2
271.869995    2
272.609985    2
166.690002    2
240.279999    2
             ..
133.240005    1
165.869995    1
317.869995    1
171.160004    1
181.690002    1
Name: count, Length: 925, dtype: int64

In [13]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)


In [14]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)


C:\Users\quaku\anaconda3\envs\mlenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\quaku\anaconda3\envs\mlenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


ValueError: Unknown label type: 'continuous'

In [12]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
rf_predictions = rf_model.predict(X_test_scaled)

C:\Users\quaku\anaconda3\envs\mlenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\quaku\anaconda3\envs\mlenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


ValueError: Unknown label type: 'continuous'